In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import collections
import logging
import json
import re

import torch
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertModel

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [2]:
# layer_indexes = [-1,-2]
pretrain_params = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(pretrain_params)

In [3]:
model = BertModel.from_pretrained(pretrain_params)

In [6]:
# model.to(device)
# model.eval()
# if n_gpu > 1:
#     model = torch.nn.DataParallel(model)

In [7]:
# sentence_root = '/p300/flickr30k_images/flickr30k_anno/sent_anno.json'
# sent_anno = json.load(open(sentence_root, 'r'))
# MAX_LEN = 88 # 86+2
# max_len = 0
# ret = {}
# for im_id, v in sent_anno.items():
#     tokens = []
#     for sent in v:
#         seq = sent['sentence']
#         seq = "[CLS] " + seq + " [SEP]"
#         tokenized_text = tokenizer.tokenize(seq)
#         if len(tokenized_text) <= MAX_LEN:
#             tokenized_text += ['[PAD]' for _ in range(MAX_LEN - len(tokenized_text))]
#         indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
#         tokens_tensor = torch.tensor(indexed_tokens).cuda()
#         tokens.append(tokens_tensor)
#     tokens_tensor = torch.stack(tokens)
    
#     with torch.no_grad():
#         encoded_layers, _ = model(tokens_tensor, None)
#     ret[im_id] = encoded_layers[-1]

In [32]:
torch.norm?

In [49]:
def dist(vec1, vec2):
    return (vec1*vec2).sum()

In [16]:
seqs = ['man', 'woman', 'kid', 'son', 'mother', 'cat', 'dog', 'duck', 'tree', 'flower', 'building']
embeds = []
for seq in seqs:
    seq = "[CLS] " + seq + " [SEP]"
    tokenized_text = tokenizer.tokenize(seq)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor(indexed_tokens)
    tokens_tensor = torch.stack([tokens_tensor])
    encoded_layers, _ = model(tokens_tensor, None)
    embeds.append(encoded_layers[-1][0][1])

In [39]:
print(dist(embeds[0], embeds[-3]))

tensor(0.6691, grad_fn=<DivBackward0>)


In [37]:
print(dist(embeds[0], embeds[3]))

tensor(0.6236, grad_fn=<DivBackward0>)


In [40]:
from skipthoughts import UniSkip

In [41]:
dir_st = 'data/skip-thoughts'
vocab = ['A', 'lot', 'of', 'beautiful', 'man', 'woman', 'kid', 'son', 'mother', 'cat', 
         'dog', 'duck', 'tree', 'flower', 'building']
uniskip = UniSkip(dir_st, vocab)

/root/venv/pose/lib/python3.6/site-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [43]:
input = torch.LongTensor([
    [1,5,0,0],
    [1,2,3,6],
    [1,10,0,0],
    [1,11,0,0],
])

In [46]:
embeds = uniskip(input, lengths=[2,4,2,2])

In [50]:
print(dist(embeds[0], embeds[1]))

tensor(38.1826, grad_fn=<SumBackward0>)


In [51]:
print(dist(embeds[0], embeds[2]))

tensor(36.2752, grad_fn=<SumBackward0>)


In [52]:
print(dist(embeds[2], embeds[3]))

tensor(39.9834, grad_fn=<SumBackward0>)


In [8]:
seq = 'A man walked across a street holding a basket on top of his head filled with fruit and vegetables with one hand , while holding bags of grapes in his right hand .'
seq = "[CLS] " + seq + " [SEP]"
print(seq)

[CLS] A man walked across a street holding a basket on top of his head filled with fruit and vegetables with one hand , while holding bags of grapes in his right hand . [SEP]


In [9]:
tokenized_text = tokenizer.tokenize(seq)
print(tokenized_text)

['[CLS]', 'a', 'man', 'walked', 'across', 'a', 'street', 'holding', 'a', 'basket', 'on', 'top', 'of', 'his', 'head', 'filled', 'with', 'fruit', 'and', 'vegetables', 'with', 'one', 'hand', ',', 'while', 'holding', 'bags', 'of', 'grapes', 'in', 'his', 'right', 'hand', '.', '[SEP]']


In [10]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
print(indexed_tokens)
print(len(indexed_tokens))

[101, 1037, 2158, 2939, 2408, 1037, 2395, 3173, 1037, 10810, 2006, 2327, 1997, 2010, 2132, 3561, 2007, 5909, 1998, 11546, 2007, 2028, 2192, 1010, 2096, 3173, 8641, 1997, 16575, 1999, 2010, 2157, 2192, 1012, 102]
35


In [11]:
tokens_tensor = torch.tensor(indexed_tokens)
tokens_tensor = torch.stack([tokens_tensor])

In [12]:
encoded_layers, _ = model(tokens_tensor, None)

In [13]:
encoded_layers[-1].shape

torch.Size([1, 35, 768])